In [1]:
import pandas as pd
import numpy as np
#articles.csv available  at https://www.kaggle.com/marlesson/news-of-the-site-folhauol
articles = pd.read_csv('articles.csv')
articles.head()

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


In [2]:
articles.category.value_counts(normalize=True)

poder                           0.131826
colunas                         0.129432
mercado                         0.125529
esporte                         0.118106
mundo                           0.102542
cotidiano                       0.101567
ilustrada                       0.097843
opiniao                         0.027087
paineldoleitor                  0.024010
saopaulo                        0.023675
tec                             0.013529
tv                              0.012822
educacao                        0.012679
turismo                         0.011392
ilustrissima                    0.008446
ciencia                         0.007991
equilibrioesaude                0.007854
sobretudo                       0.006327
bbc                             0.005866
folhinha                        0.005244
empreendedorsocial              0.005034
comida                          0.004957
asmais                          0.003280
ambiente                        0.002939
seminariosfolha 

In [5]:
articles.category.value_counts()

poder                           22022
colunas                         21622
mercado                         20970
esporte                         19730
mundo                           17130
cotidiano                       16967
ilustrada                       16345
opiniao                          4525
paineldoleitor                   4011
saopaulo                         3955
tec                              2260
tv                               2142
educacao                         2118
turismo                          1903
ilustrissima                     1411
ciencia                          1335
equilibrioesaude                 1312
sobretudo                        1057
bbc                               980
folhinha                          876
empreendedorsocial                841
comida                            828
asmais                            548
ambiente                          491
seminariosfolha                   379
serafina                          334
o-melhor-de-

In [36]:
articles.loc[articles.category=='colunas'].title.iloc[:10].values

array(['50 anos depois da morte, o Che é apenas uma foto na camiseta',
       'Afinal, nosso voto tem preço?',
       'Cientista dá psicodélico para minicérebros, e eles gostam',
       'Reestruturação da Sabesp deverá ser levada à Justiça por setor de limpeza',
       'Preservar passado é um dos grandes desafios dos processos de revitalização',
       'Não à Copa sem Lionel Messi',
       'UNE processa Lollapalooza por suspeita de burlar lei da meia entrada',
       "Esquerda pode tomar democratas, que vivem 'cabo-de-guerra', diz 'NYT'",
       'Putin encontra no Oriente Médio maior triunfo diplomático',
       'O candidato da situação'], dtype=object)

Let's keep only the 10'st commom headline categories.

In [2]:
categories = articles.category.value_counts().index[:10]
filtered_articles = articles.loc[articles.category.isin(categories)]
filtered_articles.shape

(147277, 6)

In [44]:
filtered_articles.category.value_counts(normalize=True)

poder             0.149528
colunas           0.146812
mercado           0.142385
esporte           0.133965
mundo             0.116311
cotidiano         0.115205
ilustrada         0.110981
opiniao           0.030724
paineldoleitor    0.027234
saopaulo          0.026854
Name: category, dtype: float64

In [44]:
import sentencepiece as spm
import re

In [45]:
titles = articles.title.apply(lambda x: re.sub('[\t]', '', x)).str.cat(sep=' ')

In [46]:
titles_file = open('titles-classification.txt', 'w+')
titles_file.write(titles)
titles_file.close()

In [47]:
spm.SentencePieceTrainer.train('--input=titles-classification.txt --model_prefix=classification --user_defined_symbols=<PAD> --vocab_size=8000')

True

In [48]:
sp = spm.SentencePieceProcessor()
sp.load('classification.model')

True

In [51]:
X = filtered_articles.title.apply(lambda x: sp.encode_as_ids(x))

In [52]:
def build_target(category):
    zeros = np.zeros(10)
    zeros[np.where(categories==category)] = 1
    return zeros

In [53]:
np.where(categories=='poder')[0][0]

0

In [54]:
#y = filtered_articles.category.apply(lambda y: build_target(y))
y = filtered_articles.category.apply(lambda y: np.where(categories==y)[0][0])

In [61]:
y[:10]

0     0
1     6
2     0
3     6
4     2
5     4
6     0
7     3
8     0
10    0
Name: category, dtype: int64

In [55]:
from sklearn.model_selection import train_test_split

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [57]:
import tensorflow as tf

In [35]:
scce = tf.keras.losses.SparseCategoricalCrossentropy()

In [37]:
y_true = tf.convert_to_tensor([1, 2] )
y_pred = tf.convert_to_tensor([[0.05, 0.95, 0], [0.1, 0.8, 0.1]])
# Using 'auto'/'sum_over_batch_size' reduction type. 
scce(y_true, y_pred).numpy() 

1.1769392

In [58]:
X_train_dataset = tf.data.Dataset.from_generator(lambda: iter(X_train.to_list()), tf.int32)
y_train_dataset = tf.data.Dataset.from_tensor_slices(y_train.to_list())

In [59]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [68]:
train_dataset = tf.data.Dataset.zip((X_train_dataset, y_train_dataset))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None],[]), padding_values=(3, 0))
for i in train_dataset.take(1):
    print(i)

(<tf.Tensor: shape=(64, 27), dtype=int32, numpy=
array([[2541, 1706,  588, ...,    3,    3,    3],
       [  19,  943,  137, ...,    3,    3,    3],
       [ 113, 7168,   76, ...,    3,    3,    3],
       ...,
       [ 180,    5,    4, ...,    3,    3,    3],
       [ 124,    4, 1567, ...,    3,    3,    3],
       [  28, 1717,   10, ...,    3,    3,    3]], dtype=int32)>, <tf.Tensor: shape=(64,), dtype=int32, numpy=
array([1, 3, 7, 5, 8, 2, 2, 2, 3, 0, 6, 3, 2, 2, 6, 2, 2, 6, 2, 2, 2, 2,
       5, 3, 0, 4, 6, 5, 8, 6, 3, 0, 2, 0, 1, 2, 2, 2, 1, 3, 3, 4, 1, 3,
       1, 0, 5, 2, 7, 4, 4, 2, 3, 4, 1, 4, 0, 4, 0, 3, 2, 2, 1, 1],
      dtype=int32)>)


In [69]:
X_test_dataset = tf.data.Dataset.from_generator(lambda: iter(X_test.to_list()), tf.int32)
y_test_dataset = tf.data.Dataset.from_tensor_slices(y_test.to_list())
test_dataset = tf.data.Dataset.zip((X_test_dataset, y_test_dataset))
test_dataset = test_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None],[]), padding_values=(3, 0))

In [62]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(sp.get_piece_size(), 256),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [63]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [64]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 256)         2048000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               164352    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 2,221,258
Trainable params: 2,221,258
Non-trainable params: 0
_________________________________________________________________


In [70]:
model.fit(train_dataset, epochs=1,
                    validation_data=test_dataset, 
                    validation_steps=30)

1611/1611 [==============================] - 74s 46ms/step - loss: 0.7745 - accuracy: 0.7549 - val_loss: 0.7567 - val_accuracy: 0.7609


In [76]:
model.predict([X[1]])

array([[-0.3974078,  0.5479887, -0.2053893,  0.4809031, -0.8789332,
         0.4551736,  2.9599698, -2.056802 , -1.2214806,  1.6204202]],
      dtype=float32)

In [77]:
y[1]

6

In [65]:
sp.id_to_piece(1)
sp.get_piece_size()

8000